# DeltaStream Gold Layer
---
Will perform the following in this stage:
- Read silver layer data
- Create seperate aggregate views
  - How average speed and vertical speed change with wind speed
  - Speed and altitude changes over time by aircraft
  - Number of planes on the ground, taking off/landing, in the air
  - Whole data
- Create gold delta tables for all
  - Perform upserts
- Send to S3

In [0]:
from pyspark.sql.functions import (from_json, col, when, from_unixtime, 
                                   round, schema_of_json, lit, date_format, 
                                   row_number, abs, to_timestamp)
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException

In [0]:
silver_data_df = spark.sql("SELECT * FROM delta_silver")

In [0]:
display(silver_data_df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS speed_table (
  avg_speed DOUBLE,
  avg_vert_speed DOUBLE,
  wind_speed DOUBLE
) USING DELTA;


CREATE TABLE IF NOT EXISTS changes_over_time (
  callsign STRING,
  min_time_date STRING,
  max_time_date STRING,
  min_time DATE,
  max_time DATE,
  time_diff_seconds INT,
  old_altitude DOUBLE,
  new_altitude DOUBLE,
  altitude_change_rate DOUBLE,
  min_speed DOUBLE,
  max_speed DOUBLE
) USING DELTA;


CREATE TABLE IF NOT EXISTS airplane_counts (
  num_on_ground DOUBLE, 
  num_flying DOUBLE, 
  num_takeoff DOUBLE, 
  num_landing DOUBLE
) USING DELTA;

#### How average speed and vertical speed change with wind speed
---

In [0]:
speed_table_df = spark.sql("""
                           
    SELECT AVG(speed) as avg_speed, AVG(vertical_speed) as avg_vert_speed, wind_speed
    FROM delta_silver
    WHERE on_ground = 'false'
    GROUP BY wind_speed
    HAVING wind_speed > 0
    ORDER BY wind_speed DESC

""")
display(speed_table_df)
speed_table_df.createOrReplaceTempView("speed_table_temp")

#### Speed and altitude changes over time by aircraft
---

In [0]:
changes_over_time_df = spark.sql(""" 

    WITH min_max_times AS (
        SELECT
            callsign,
            MIN(time_position) AS min_time_date,
            MAX(time_position) AS max_time_date,
            MIN(to_timestamp(time_position, 'MM-dd-yyyy HH:mm:ss')) AS min_time,
            MAX(to_timestamp(time_position, 'MM-dd-yyyy HH:mm:ss')) AS max_time,
            MIN(speed) AS min_speed,
            MAX(speed) AS max_speed
        FROM delta_silver
        GROUP BY callsign
    ),

    d_old_dedup AS (
        SELECT *
        FROM (
            SELECT *,
                ROW_NUMBER() OVER (PARTITION BY callsign, time_position ORDER BY altitude DESC) AS rn
            FROM delta_silver
        ) t
        WHERE rn = 1
    ),

    d_new_dedup AS (
        SELECT *
        FROM (
            SELECT *,
                ROW_NUMBER() OVER (PARTITION BY callsign, time_position ORDER BY altitude DESC) AS rn
            FROM delta_silver
        ) t
        WHERE rn = 1
    )

    SELECT
        m.callsign,
        m.min_time_date,
        m.max_time_date,
        m.min_time,
        m.max_time,
        UNIX_TIMESTAMP(m.max_time) - UNIX_TIMESTAMP(m.min_time) AS time_diff_seconds,
        CAST(d_old.altitude AS double) AS old_altitude,
        CAST(d_new.altitude AS double) AS new_altitude,
        (new_altitude - old_altitude) / (time_diff_seconds + 0.0001) AS altitude_change_rate,
        m.min_speed,
        m.max_speed
    FROM min_max_times m
    LEFT JOIN d_old_dedup d_old
        ON m.callsign = d_old.callsign AND m.min_time_date = d_old.time_position
    LEFT JOIN d_new_dedup d_new
        ON m.callsign = d_new.callsign AND m.max_time_date = d_new.time_position;

""")
display(changes_over_time_df)
changes_over_time_df.createOrReplaceTempView("changes_over_time_temp")

#### Number of planes on the ground, taking off/landing, in the air
---

In [0]:
airplane_counts_df = spark.sql(""" 

  WITH airplanes_latest AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY callsign ORDER BY time_position DESC) AS rn
    FROM delta_silver
  )

  SELECT
    COUNT(CASE WHEN CAST(altitude AS double) = 0 THEN 1 END) AS num_on_ground,
    COUNT(CASE WHEN CAST(altitude AS double) > 2000 THEN 1 END) AS num_flying,
    COUNT(CASE WHEN CAST(altitude AS double) > 0 AND CAST(altitude AS double) <= 2000 AND vertical_speed > 0 THEN 1 END) AS num_takeoff,
    COUNT(CASE WHEN CAST(altitude AS double) > 0 AND CAST(altitude AS double) <= 2000 AND vertical_speed < 0 THEN 1 END) AS num_landing
  FROM airplanes_latest
  WHERE rn = 1;

""")
display(airplane_counts_df)
airplane_counts_df.createOrReplaceTempView("airplane_counts_temp")

#### Save aggregates to delta tables
---

In [0]:
%sql
INSERT INTO speed_table
SELECT * FROM speed_table_temp;


MERGE INTO changes_over_time AS TARGET
USING changes_over_time_temp AS SOURCE
ON TARGET.callsign = SOURCE.callsign
WHEN MATCHED THEN 
  UPDATE SET 
    target.max_time_date = source.max_time_date,
    target.max_time = source.max_time,
    target.time_diff_seconds = UNIX_TIMESTAMP(source.max_time) - UNIX_TIMESTAMP(target.min_time),
    target.new_altitude = source.new_altitude,
    target.altitude_change_rate = (source.new_altitude - target.old_altitude) / 
                                  ((UNIX_TIMESTAMP(source.max_time) - UNIX_TIMESTAMP(target.min_time)) + 0.0001),
    target.max_speed = source.max_speed
WHEN NOT MATCHED THEN INSERT *;



INSERT INTO airplane_counts
SELECT * FROM airplane_counts_temp;